In [15]:
import torch
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random

In [4]:
# matches our graphed gradients when implementing micrograd from scratch
x1 = torch.Tensor([2.0]).double()                ; x1.requires_grad = True
x2 = torch.Tensor([0.0]).double()                ; x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double()               ; w1.requires_grad = True
w2 = torch.Tensor([1.0]).double()                ; w2.requires_grad = True
b = torch.Tensor([6.8813735870195432]).double()  ; b.requires_grad = True
n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()

print('---')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())

0.7071066904050358
---
x2 0.5000001283844369
w2 0.0
x1 -1.5000003851533106
w1 1.0000002567688737


In [3]:
torch.Tensor([[1, 2, 3], [4, 5, 6]])

tensor([[1., 2., 3.],
        [4., 5., 6.]])

In [6]:
x1.grad

tensor([-1.5000], dtype=torch.float64)

In [7]:
o.item()

0.7071066904050358

In [67]:
# allow for addition of constants plus value objects
# ex: a = Value(2.0)
#     a + 1

class Value:
    
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label
        
        
    def __repr__(self):
        return f"Value(data={self.data})"
    
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward

        return out
    
    
    def __neg__(self):
        return self * -1
    
    
    def __sub__(self, other):
        return self + (-other)
    
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        
        return out
    
    
    def __rmul__(self, other): # other * self
        return self * other
    
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self, ), f'**{other}')
        
        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward
        
        return out
    
    
    def __truediv__(self, other):
        return self * other**-1
    
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')
        
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        
        return out
    
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')
        
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        
        return out
    
    
    def backward(self):
        
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [102]:
class Neuron:
    
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
        
    def __call__(self, x):
        # w * x + b
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]
    
class Layer:
    
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
        
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
        
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
        #params = []
        #for neuron in self.neurons:
        #    ps = neuron.parameters()
        #    params.extend(ps)
        #return params
        
class MLP:
    
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
        
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [173]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])  # 3 input neurons, into two layers of 4, and 1 output neuron
n(x) # forward pass of multi layer perceptron

Value(data=0.6661958020788785)

In [174]:
n.parameters() # all of the weights and biases inside the MLP

[Value(data=0.896130422368818),
 Value(data=0.20769674622905265),
 Value(data=0.1485489367438786),
 Value(data=0.2909305199544028),
 Value(data=0.8645981081266256),
 Value(data=0.9474079772106976),
 Value(data=0.5612266347796129),
 Value(data=0.019048446096525007),
 Value(data=-0.2916225445035572),
 Value(data=0.28102416572565225),
 Value(data=-0.4356700547953818),
 Value(data=0.9664279670670877),
 Value(data=-0.47607379592025945),
 Value(data=-0.6913514485722017),
 Value(data=0.4396528035314051),
 Value(data=0.01299648222536165),
 Value(data=-0.4762178346236092),
 Value(data=-0.804739705362499),
 Value(data=-0.27676381255480664),
 Value(data=-0.549610035989526),
 Value(data=-0.3568875455357201),
 Value(data=0.3388231593445177),
 Value(data=0.38391596015736984),
 Value(data=0.6416790514617219),
 Value(data=-0.9961448228158007),
 Value(data=-0.9135616078364512),
 Value(data=0.4673820169484937),
 Value(data=0.2784093966809067),
 Value(data=-0.3550910574538064),
 Value(data=0.872905024339

In [253]:
# example dataset
# this dataset has 4 examples, 4 possible inputs:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]

# desired outputs
# saying this should be the output for each corresponding input (4 options)
ys = [1.0, -1.0, -1.0, 1.0]

In [254]:
ypred = [n(x) for x in xs]
ypred

[Value(data=1.0), Value(data=-1.0), Value(data=-1.0), Value(data=1.0)]

In [177]:
loss_list = [(yout - ygt)**2 for ygt, yout in zip(ys, ypred)]
loss = Value(0.0)
for i in loss_list: 
    print(i)
    loss += i
loss

Value(data=0.11142524254976324)
Value(data=3.6470931844314918)
Value(data=3.3888825914937257)
Value(data=0.10830742580990207)


Value(data=7.255708444284882)

In [178]:
loss.backward()

In [179]:
n.layers[0].neurons[0].w[0].grad

-0.08854019046144276

In [180]:
n.layers[0].neurons[0].w[0].data

0.896130422368818

In [244]:
# forward pass
ypred = [n(x) for x in xs]

loss_list = [(yout - ygt)**2 for ygt, yout in zip(ys, ypred)]
loss = Value(0.0)
for i in loss_list: 
    print(i)
    loss += i
loss

Value(data=1.553081012909185e-05)
Value(data=0.014116434925631234)
Value(data=0.00545553015383858)
Value(data=1.7854847091108306e-05)


Value(data=0.01960535073669001)

In [245]:
# backward pass
loss.backward()

In [246]:
# nudge (modify to get closer to target)
for p in n.parameters():
    p.data += -0.01 * p.grad # want to travel in opposite direction of gradient to minimize loss

In [247]:
ypred
# Now that the loss is very low, the output values almost match the target output ys

[Value(data=0.9960590851152186),
 Value(data=-0.8811873957627759),
 Value(data=-0.9261384392675149),
 Value(data=0.9957745003737891)]

In [248]:
n.parameters()
# these are the weights and biases of the MLP that are being used to adjust to reach this loss

[Value(data=0.48020607119603076),
 Value(data=-0.08465864992077649),
 Value(data=1.4883492005950762),
 Value(data=0.21285633429543402),
 Value(data=0.8744311138512532),
 Value(data=0.6489052333744061),
 Value(data=0.9936469304215929),
 Value(data=-0.09864017440740246),
 Value(data=-0.504313058689415),
 Value(data=0.5593936628729803),
 Value(data=-0.6698746904605593),
 Value(data=0.932272391605097),
 Value(data=-0.3172493342909981),
 Value(data=-0.28382432131066004),
 Value(data=1.0772362838210006),
 Value(data=0.4320026353903077),
 Value(data=-0.42407553825597905),
 Value(data=-0.7320246658993985),
 Value(data=-0.6331566791035429),
 Value(data=-0.3561235903014128),
 Value(data=-0.3106647093489745),
 Value(data=1.7410850836761511),
 Value(data=1.8888585368660638),
 Value(data=0.6879204618206212),
 Value(data=-1.6951300327506547),
 Value(data=0.6356015686013462),
 Value(data=1.0503160164102476),
 Value(data=0.9934376184767668),
 Value(data=-2.4351674200405315),
 Value(data=2.143751945367

In [255]:
# reset neural net
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])  # 3 input neurons, into two layers of 4, and 1 output neuron
n(x) # forward pass of multi layer perceptron

Value(data=0.5350786600275635)

In [256]:
# data stays the same
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0]

ypred = [n(x) for x in xs]
ypred

[Value(data=0.5350786600275635),
 Value(data=0.7078196020751478),
 Value(data=-0.24768800313367445),
 Value(data=0.672108268907658)]

In [261]:
# no more manual adjusting

# gradient descent:
for k in range(20):
    
    # forward pass
    ypred = [n(x) for x in xs]

    loss_list = [(yout - ygt)**2 for ygt, yout in zip(ys, ypred)]
    loss = Value(0.0)
    for i in loss_list: loss += i
    
    # backward pass
    # make sure to reset gradients on every pass to avoid bug
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()
    
    # update
    for p in n.parameters():
        p.data += -0.05 * p.grad
    
    print(k, loss.data)

0 0.042282670218117054
1 0.04081737601449683
2 0.0394419153456522
3 0.038148596793736216
4 0.03693056134016767
5 0.035781674634878254
6 0.03469643536029679
7 0.0336698969815352
8 0.03269760067855878
9 0.031775517657955436
10 0.03089999936375794
11 0.03006773436581805
12 0.029275710913726397
13 0.028521184314463342
14 0.02780164843084027
15 0.02711481071157406
16 0.026458570257449795
17 0.025830998505337335
18 0.02523032217591267
19 0.024654908184249466


In [262]:
ypred

[Value(data=0.9352039947486268),
 Value(data=-0.891586159348699),
 Value(data=-0.9463042158468494),
 Value(data=0.923713777081795)]